<a href="https://colab.research.google.com/github/37stu37/GNS_GAN/blob/master/ctgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import**
---



In [0]:
import pandas as pd
import pathlib as Path

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN

/content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN


In [0]:
outputDir = '../../02_output'

In [0]:
data = pd.read_parquet(outputDir + '/smallMonteCarlo2500runs.parquet')

**Data processing**
---



In [0]:
data.reset_index(inplace=True)
data.drop(['index', 'scenario', 'pid'], axis=1, inplace=True)

In [7]:
len(data); data.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [0]:
trainingSample = data.sample(n=7000)
discrete_columns = ['source', 'target']

**CTGAN**

In [0]:
%%capture
!pip install ctgan 

In [0]:
%%time
from ctgan import CTGANSynthesizer

ctgan = CTGANSynthesizer()
ctgan.fit(trainingSample, discrete_columns)

**Generate synthetic data**

In [0]:
samples = ctgan.sample(1000)